In [1]:
# Silence noisy deprecations across workers (e.g., pkg_resources)
import os, warnings
os.environ.setdefault(
    "PYTHONWARNINGS",
    "ignore:pkg_resources is deprecated as an API:UserWarning"
)
warnings.filterwarnings("ignore", message=r"pkg_resources is deprecated as an API", category=UserWarning)

import numpy as np
import pandas as pd
from collections import defaultdict

from sklearn.model_selection import StratifiedKFold, cross_val_predict, learning_curve, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, SplineTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (
    roc_auc_score, average_precision_score, f1_score,
    accuracy_score, brier_score_loss
)
from sklearn.base import clone, BaseEstimator, ClassifierMixin
from tqdm.auto import tqdm

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Load combined_df from CSV in /app/data/processed/
combined_df = pd.read_csv("/app/data/processed/combined_df.csv")
feature_cols = [c for c in combined_df.columns if c.startswith("original_")]
X = combined_df[feature_cols].copy()
y = combined_df["target"].astype(int).values

print(f"Rows: {len(y)}, Radiomics features: {len(feature_cols)}")
print("Class balance (target):\n", pd.Series(y).value_counts(normalize=True).rename("pct").round(3))

Rows: 1905, Radiomics features: 102
Class balance (target):
 0    0.504
1    0.496
Name: pct, dtype: float64


/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 1) Common preprocessors
preprocess_linear = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("scale", StandardScaler())
])

preprocess_tree = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    # no scaling for tree-based models
])

preprocess_spline = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("scale", StandardScaler()),
    ("spline", SplineTransformer(
        degree=3,
        n_knots=6,
        include_bias=False
    )),
])

# deep preprocessor (impute + scale)
preprocess_deep = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("scale", StandardScaler()),
])

# 2) Models to compare
models = {
    "LogReg_L2": Pipeline([
        ("prep", preprocess_linear),
        ("clf", LogisticRegression(
            C=1.0, penalty="l2", solver="liblinear",
            class_weight="balanced", random_state=RANDOM_STATE, max_iter=1000
        ))
    ]),
    "LogReg_spline": Pipeline([
        ("prep", preprocess_spline),
        ("clf", LogisticRegression(
            C=1.0, penalty="l2", solver="liblinear",
            class_weight="balanced", random_state=RANDOM_STATE, max_iter=1000
        ))
    ]),
    "SVM_linear": Pipeline([
        ("prep", preprocess_linear),
        ("clf", SVC(
            kernel="linear", C=1.0, probability=True,
            class_weight="balanced", random_state=RANDOM_STATE
        ))
    ]),
    "SVM_linear_spline": Pipeline([
        ("prep", preprocess_spline),
        ("clf", SVC(
            kernel="linear", C=1.0, probability=True,
            class_weight="balanced", random_state=RANDOM_STATE
        ))
    ]),
    "RandomForest": Pipeline([
        ("prep", preprocess_tree),
        ("clf", RandomForestClassifier(
            n_estimators=300, max_depth=None, min_samples_leaf=2,
            n_jobs=-1, class_weight="balanced", random_state=RANDOM_STATE
        ))
    ]),
    "GradBoost": Pipeline([
        ("prep", preprocess_tree),
        ("clf", GradientBoostingClassifier(
            learning_rate=0.05, n_estimators=300, max_depth=3,
            random_state=RANDOM_STATE
        ))
    ]),
}

# 2a) Add FT-Transformer (optional; requires torch + skorch)
_ft_ok = True
try:
    import torch
    import torch.nn as nn
    from skorch import NeuralNetClassifier
except Exception as _e:
    _ft_ok = False
    print("[Info] Skipping FTTransformer (torch/skorch missing):", _e)

if _ft_ok:
    class FTTransformerModule(nn.Module):
        def __init__(self, n_features, d_token=64, n_heads=8, n_layers=4, p_drop=0.1, n_classes=2):
            super().__init__()
            self.n_features = n_features
            self.proj = nn.Linear(1, d_token)
            self.feature_embed = nn.Parameter(torch.randn(n_features, d_token))
            enc_layer = nn.TransformerEncoderLayer(
                d_model=d_token, nhead=n_heads, dim_feedforward=4*d_token,
                dropout=p_drop, activation="gelu", batch_first=True, norm_first=True
            )
            # silence nested-tensor warning, no behavior change
            try:
                self.encoder = nn.TransformerEncoder(enc_layer, num_layers=n_layers, enable_nested_tensor=False)
            except TypeError:
                self.encoder = nn.TransformerEncoder(enc_layer, num_layers=n_layers)
            self.cls_head = nn.Sequential(
                nn.LayerNorm(d_token),
                nn.Linear(d_token, max(32, d_token // 2)),
                nn.GELU(),
                nn.Dropout(p_drop),
                nn.Linear(max(32, d_token // 2), n_classes)
            )

        def forward(self, X):
            B, F = X.shape
            x = X.unsqueeze(-1)                      # (B,F,1)
            x = self.proj(x) + self.feature_embed.unsqueeze(0)  # (B,F,d)
            h = self.encoder(x)                      # (B,F,d)
            h = h.mean(dim=1)                        # global average over features
            return self.cls_head(h)                  # logits

    class FTTransformerEstimator(BaseEstimator, ClassifierMixin):
        def __init__(
            self,
            n_epochs=60,
            batch_size=256,
            lr=1e-3,
            d_token=64,
            n_heads=8,
            n_layers=4,
            p_drop=0.1,
            device=None,
            random_state=42,
        ):
            self.n_epochs = n_epochs
            self.batch_size = batch_size
            self.lr = lr
            self.d_token = d_token
            self.n_heads = n_heads
            self.n_layers = n_layers
            self.p_drop = p_drop
            self.device = device
            self.random_state = random_state
            self._net = None
            self._n_features = None
            self.classes_ = None

        def _build(self, n_features):
            module = FTTransformerModule(
                n_features=n_features,
                d_token=self.d_token,
                n_heads=self.n_heads,
                n_layers=self.n_layers,
                p_drop=self.p_drop,
                n_classes=2,
            )
            dev = self.device or ("cuda" if torch.cuda.is_available() else "cpu")
            net = NeuralNetClassifier(
                module,
                max_epochs=self.n_epochs,
                lr=self.lr,
                batch_size=self.batch_size,
                optimizer=torch.optim.AdamW,
                iterator_train__shuffle=True,
                train_split=None,
                device=dev,
                verbose=0,
            )
            return net

        def fit(self, X, y):
            torch.manual_seed(self.random_state)
            self._n_features = X.shape[1]
            self._net = self._build(self._n_features)
            self._net.fit(X.astype("float32"), y.astype("int64"))
            self.classes_ = np.array(sorted(np.unique(y)))
            return self

        def predict_proba(self, X):
            self._net.module_.eval()
            with torch.no_grad():
                Xt = torch.as_tensor(X.astype("float32"), device=self._net.device)
                logits = self._net.module_(Xt)
                if logits.ndim == 1 or logits.shape[-1] == 1:
                    p1 = torch.sigmoid(logits.view(-1))
                    proba = torch.stack([1 - p1, p1], dim=1)
                else:
                    proba = torch.softmax(logits, dim=1)
            return np.clip(proba.detach().cpu().numpy(), 0.0, 1.0)

        def predict(self, X):
            proba = self.predict_proba(X)
            return (proba[:, 1] >= 0.5).astype(int)

    models["FTTransformer"] = Pipeline([
        ("prep", preprocess_deep),
        ("clf", FTTransformerEstimator(
            n_epochs=60, batch_size=256, lr=1e-3,
            d_token=64, n_heads=8, n_layers=4, p_drop=0.1,
            random_state=RANDOM_STATE,
        ))
    ])

# 3) CV scheme - stratified KFold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

def evaluate_oof(model, X, y, cv):
    """
    Get out-of-fold probabilities and compute metrics.
    """
    oof_proba = cross_val_predict(model, X, y, cv=cv, method="predict_proba", n_jobs=-1)[:, 1]
    oof_pred  = (oof_proba >= 0.5).astype(int)
    metrics = {
        "ROC_AUC": roc_auc_score(y, oof_proba),
        "PR_AUC": average_precision_score(y, oof_proba),
        "F1": f1_score(y, oof_pred),
        "ACC": accuracy_score(y, oof_pred),
        "Brier": brier_score_loss(y, oof_proba),
    }
    return oof_proba, oof_pred, metrics

def bootstrap_ci(y, oof_proba, oof_pred, B=300, seed=RANDOM_STATE, show_progress=False):
    """
    Percentile 95% CIs around metrics using bootstrap resamples of OOF predictions.
    Keeps data amount fixed; no extra holdout.
    """
    rng = np.random.default_rng(seed)
    n = len(y)
    stats = defaultdict(list)
    iterator = range(B)
    if show_progress:
        iterator = tqdm(iterator, desc="Bootstrapping", leave=False)
    for _ in iterator:
        idx = rng.integers(0, n, n)
        yt = y[idx]
        pt = oof_proba[idx]
        ht = (pt >= 0.5).astype(int)
        stats["ROC_AUC"].append(roc_auc_score(yt, pt))
        stats["PR_AUC"].append(average_precision_score(yt, pt))
        stats["F1"].append(f1_score(yt, ht))
        stats["ACC"].append(accuracy_score(yt, ht))
        stats["Brier"].append(brier_score_loss(yt, pt))
    ci = {k: (np.percentile(v, 2.5), np.percentile(v, 97.5)) for k, v in stats.items()}
    return ci

# --- OOF comparison (progress per model) ---
rows_oof = []
print("\nOOF (5-fold) — models:")
for name, pipe in tqdm(models.items(), desc="OOF models"):
    pipe_cloned = clone(pipe)
    oof_proba, oof_pred, m = evaluate_oof(pipe_cloned, X, y, cv=cv)
    rows_oof.append({"model": name, **m})
summary_oof = pd.DataFrame(rows_oof).sort_values("ROC_AUC", ascending=False)
print("\nResults on OOF (5-fold):")
display(summary_oof)

# --- Train/Test split results ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE
)

rows_test = []
print("\nTrain on train, evaluate on 20% test:")
for name, pipe in tqdm(models.items(), desc="Holdout models"):
    pipe_cloned = clone(pipe)
    pipe_cloned.fit(X_train, y_train)
    proba = pipe_cloned.predict_proba(X_test)[:, 1]
    pred = (proba >= 0.5).astype(int)
    m = {
        "ROC_AUC": roc_auc_score(y_test, proba),
        "PR_AUC": average_precision_score(y_test, proba),
        "F1": f1_score(y_test, pred),
        "ACC": accuracy_score(y_test, pred),
        "Brier": brier_score_loss(y_test, proba),
    }
    ci = bootstrap_ci(y_test, proba, pred, B=300, show_progress=True)
    rows_test.append({
        "model": name,
        **{f"{k}_mean": v for k, v in m.items()},
        **{f"{k}_ci": f"[{ci[k][0]:.3f}, {ci[k][1]:.3f}]" for k in m.keys()}
    })
summary_test = pd.DataFrame(rows_test).sort_values("ROC_AUC_mean", ascending=False)
print("\nResults on test set (20% holdout):")
display(summary_test)


OOF (5-fold) — models:


OOF models: 100%|██████████| 7/7 [01:22<00:00, 11.76s/it]


Results on OOF (5-fold):


,model,ROC_AUC,PR_AUC,F1,ACC,Brier
4,RandomForest,0.608025,0.607382,0.555008,0.573228,0.241794
5,GradBoost,0.603439,0.597918,0.552255,0.572703,0.248129
0,LogReg_L2,0.592959,0.588741,0.552360,0.566929,0.245502
2,SVM_linear,0.592362,0.581090,0.566220,0.575328,0.245216
1,LogReg_spline,0.588620,0.579955,0.536720,0.559580,0.246734
3,SVM_linear_spline,0.581779,0.561996,0.547581,0.553281,0.245294
6,FTTransformer,0.515719,0.520613,0.285283,0.502887,0.367185



Train on train, evaluate on 20% test:


Holdout models: 100%|██████████| 7/7 [00:29<00:00,  4.25s/it]


Results on test set (20% holdout):


,model,ROC_AUC_mean,PR_AUC_mean,F1_mean,ACC_mean,Brier_mean,ROC_AUC_ci,PR_AUC_ci,F1_ci,ACC_ci,Brier_ci
6,FTTransformer,0.627039,0.626281,0.000000,0.503937,0.496052,"[0.565, 0.683]","[0.557, 0.705]","[0.000, 0.000]","[0.459, 0.554]","[0.446, 0.541]"
4,RandomForest,0.601742,0.601393,0.564644,0.566929,0.243901,"[0.546, 0.655]","[0.530, 0.676]","[0.505, 0.626]","[0.521, 0.617]","[0.231, 0.258]"
3,SVM_linear_spline,0.599757,0.583225,0.570694,0.561680,0.243212,"[0.534, 0.653]","[0.518, 0.661]","[0.508, 0.630]","[0.509, 0.609]","[0.237, 0.250]"
1,LogReg_spline,0.597966,0.598706,0.557641,0.566929,0.244451,"[0.538, 0.654]","[0.530, 0.673]","[0.494, 0.614]","[0.514, 0.616]","[0.233, 0.258]"
0,LogReg_L2,0.572641,0.576389,0.550265,0.553806,0.249579,"[0.508, 0.628]","[0.513, 0.654]","[0.488, 0.610]","[0.501, 0.609]","[0.237, 0.264]"
5,GradBoost,0.570354,0.580326,0.528796,0.527559,0.259160,"[0.516, 0.625]","[0.510, 0.654]","[0.466, 0.583]","[0.474, 0.576]","[0.243, 0.276]"
2,SVM_linear,0.567075,0.560407,0.554667,0.561680,0.246637,"[0.503, 0.628]","[0.497, 0.637]","[0.492, 0.613]","[0.509, 0.614]","[0.241, 0.253]"
